In [ ]:
import rasterio, numpy as np, matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider


In [ ]:

# 1. Paths to your rasters (ML map + distance layers + binary mask)

ML_path =
roads_path =
power_path =
rail_path =
developed_mask_path =

paths = [ML_path, roads_path, power_path, rail_path, developed_mask_path]


In [ ]:

# 2. Read them all into numpy arrays
arrays = []
for p in paths:
    with rasterio.open(p) as src:
        arrays.append(src.read(1).astype(float))
        meta = src.meta  # we just need one for extent

# 3. Normalize each to 0–1
norm = [(a - np.nanmin(a)) / (np.nanmax(a) - np.nanmin(a)) for a in arrays]

# 4. Precompute map extent
t = meta["transform"]
xmin, ymax = t[2], t[5]
xmax = xmin + t[0] * meta["width"]
ymin = ymax + t[4] * meta["height"]
extent = (xmin, xmax, ymin, ymax)


# 5. Define plotting function
def show_map(**weights):
    combo = np.zeros_like(norm[0])
    for i, (w) in enumerate(weights.values()):
        combo += w * norm[i]
    plt.figure(figsize=(6, 5))
    plt.imshow(combo, extent=extent, origin="upper", cmap="viridis")
    plt.colorbar(label="Weighted sum")
    plt.axis("off")
    plt.title("Interactive Linear Combination")


# 6. Build sliders and hook them up
sliders = {
    f"w_{i}": FloatSlider(
        min=0, max=1, step=0.01, value=0.2, description=paths[i].split(".")[0]
    )
    for i in range(len(paths))
}
interact(show_map, **sliders)